In [1]:
import pandas as pd
import numpy as np
import torch
import torchvision.transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.nn as nn
from torch.optim import Adam
from torch.nn import BCEWithLogitsLoss
from yolov3.models import yolo

c:\Users\Kenneth\anaconda3\envs\aiapminiprojfinal\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
test_model = yolo.Model(cfg="yolov3/models/yolov3.yaml")


                 from  n    params  module                                  arguments                     
  0                -1  1       928  models.common.Conv                      [3, 32, 3, 1]                 
  1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]                
  2                -1  1     20672  models.common.Bottleneck                [64, 64]                      
  3                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
  4                -1  2    164608  models.common.Bottleneck                [128, 128]                    
  5                -1  1    295424  models.common.Conv                      [128, 256, 3, 2]              
  6                -1  8   2627584  models.common.Bottleneck                [256, 256]                    
  7                -1  1   1180672  models.common.Conv                      [256, 512, 3, 2]              
  8                -1  8  10498048  

In [3]:
device =  torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device ="cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")
torch.cuda.is_available()

Using cuda:0 device


True

In [4]:
TRANSFORMS = torchvision.transforms.Compose([
    # torchvision.transforms.ColorJitter(brightness=[0.5,1],contrast=[0.5,1],saturation=[0.5,1],hue=[-0.1,0.1]),
    # torchvision.transforms.RandomAffine(degrees=270,translate=(0.2,0.2)),
    # torchvision.transforms.RandomVerticalFlip(p=0.5),
    # torchvision.transforms.RandomHorizontalFlip(p=0.5),
    # torchvision.transforms.RandomRotation(degrees=270),
    torchvision.transforms.Resize((220,220)),
    torchvision.transforms.ToTensor(),
    # torchvision.transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

TEST_TRANSFORM = torchvision.transforms.Compose([
    torchvision.transforms.Resize((220,220)),
    torchvision.transforms.ToTensor(),
    # torchvision.transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

In [5]:
BINARY_OUTPUT_DIR = "images/tensorfood/part3/output/binary"
TRAIN_PATH = BINARY_OUTPUT_DIR + "/train"
TEST_PATH = BINARY_OUTPUT_DIR + "/test"
train_data = ImageFolder(root=TRAIN_PATH,transform=TRANSFORMS)
test_data = ImageFolder(root=TEST_PATH,transform=TEST_TRANSFORM)

In [6]:
train_data_loader = DataLoader(train_data,batch_size=2,shuffle=True)#Default batch size = 1
test_data_loader = DataLoader(test_data,batch_size=2)

In [7]:
criterion = BCEWithLogitsLoss()
optimiser = Adam(test_model.parameters())

In [8]:
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.inference_mode():
        maxk = max(topk)
        batch_size = target.size(0)
        if target.ndim == 2:
            target = target.max(dim=1)[1]

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target[None])

        res = []
        for k in topk:
            correct_k = correct[:k].flatten().sum(dtype=torch.float32)
            res.append(correct_k * (100.0 / batch_size))
        return res

In [21]:
test_model = test_model.to(device)
total_train_acc=[]
EPOCHS = 10
for epoch in range(EPOCHS):
    
    running_loss = 0.0
    test_model.train()
    
    for i,data in enumerate(train_data_loader,0):
        # print(i,data)
        inputs,labels = data
        labels = F.one_hot(labels,num_classes=2)
        inputs = inputs.to(device)
        labels = labels.to(device)
        # zero the parameter gradients
        optimiser.zero_grad()

        # forward + backward + optimize
        outputs = test_model(inputs)
        # print(outputs.size())
        # print(outputs)
        # outputs = torch.unsqueeze(outputs,0)
        # print(labels)
        print(len(outputs))
        print(outputs)
        # outputs = torch.Tensor(outputs)
        loss = criterion(outputs,labels.to(torch.float32))
        loss.backward()
        optimiser.step()

        # print statistics
        running_loss += loss.item()

        total_train_acc.append(accuracy(outputs.detach().cpu(),labels.detach().cpu())[0])
        
        if (i+1) % 5 == 0:    # print every 5 images
            print(f"Epoch: {epoch+1} Step {i+1}/{len(train_data_loader)} Running loss: {running_loss :.4f} ")
            # epoch_train_loss = 0.0
mean_acc = sum(total_train_acc)/len(total_train_acc)
print(mean_acc)
print('Finished Training')

3
[tensor([[[[[ 4.78518e-02,  8.36701e-03, -4.38057e-02,  ..., -4.89179e+00, -4.99436e+00, -4.84209e+00],
           [ 4.50176e-02,  8.78465e-02,  1.19100e-01,  ..., -5.06824e+00, -5.05161e+00, -4.87139e+00],
           [ 1.40998e-01,  4.64240e-02,  1.66041e-01,  ..., -4.97906e+00, -5.00712e+00, -4.85972e+00],
           ...,
           [ 3.99508e-01, -2.57223e-01,  5.69018e-02,  ..., -4.85956e+00, -4.95738e+00, -4.65421e+00],
           [ 9.97611e-02,  2.86572e-01,  5.46781e-03,  ..., -4.82642e+00, -5.14826e+00, -4.83810e+00],
           [-2.16077e-01, -9.39716e-02, -2.18159e-01,  ..., -4.96819e+00, -5.16376e+00, -4.78407e+00]],

          [[-1.13435e-01, -5.10102e-02, -1.20267e-01,  ..., -5.02091e+00, -4.97874e+00, -4.76164e+00],
           [ 5.33074e-02, -7.76237e-02,  9.85286e-02,  ..., -5.01854e+00, -5.18791e+00, -4.81742e+00],
           [ 1.03665e-01, -1.61376e-01,  1.37081e-01,  ..., -4.98643e+00, -5.19253e+00, -4.65221e+00],
           ...,
           [-4.92448e-01,  7.29230e-

AttributeError: 'list' object has no attribute 'size'